In [1]:
import os
import sys
from tqdm import tqdm
import time

import requests
from dotenv import load_dotenv

load_dotenv()
TomTomAPI_KEY = os.environ.get('TomTomAPI_KEY')

# modify sys.path for it to contain the main repo path so we can import modules such as below
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '../..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from db.db_conn import engine, session_scope, ping_db
from db.models import DimensionLocation, DimensionCountry, DimensionContinent

In [2]:
ping_db(engine)

True

In [3]:
def get_location(location: str):
    if location is None:
        raise ValueError('No location provided')
    
    try:
        response = requests.get(f"https://api.tomtom.com/search/2/geocode/{location}.json?key={TomTomAPI_KEY}")
        response.raise_for_status()  # Check for HTTP errors

        data = response.json()
        results = data.get('results')
        if results:
            position = results[0]['position']
            longitude = position['lon']
            latitude = position['lat']
            return longitude, latitude
        else:
            print(f"Could not get coordinates for {location}")
            return None, None
    except Exception as e:
        print(f"Could not get coordinates for {location} due to error: {e}")

In [4]:
with session_scope() as session:
    try:
        records = session.query(DimensionLocation).join(DimensionCountry).join(DimensionContinent).filter(DimensionLocation.latitude == None).all()
        # This corresponds to the SQL statement: SELECT * FROM dimension_location WHERE longitude IS NULL
        for record in tqdm(records, total=len(records), desc="Enriching locations..."):
            full_location = f"{record.city}, {record.country.country_name}, {record.country.continent.continent_name}"
            long, lat = get_location(full_location)
            if long is not None and lat is not None:
                record.longitude = long
                record.latitude = lat
            else:
                print(f"Could not get coordinates for {record}")
            time.sleep(1)  # Sleep for 1 second to avoid rate limiting
        session.commit()
    except Exception as e:
        session.rollback()
        print(e)
        print(f"Error occured while writing results to database. Rolling back...")

Enriching locations...: 100%|██████████| 249/249 [05:04<00:00,  1.22s/it]
